* dqn is the pioneer of deep reinforcement learning, has achieved significant sucess in various applications
* here we train a DQN agent on CartPole with Tianshou
* contrarty to existing Deep RL libraries such as RLIB
* dqn only accept a config specification of hypermaraters , network and others, tianshou orovide easy wway of construction through code-level


* agent to environemtn : action will be generated by agent and sent to environment
* environtment to agent: env.step takes action and returns a tuple of observation ,reward,done,info
* agent-environment interaction to agent training: the data generated by interaction will be stored and sent to the learner of agent
* setup
    1. verctorized environments
    2. policy (with nueral networks)
    3. collector (with buffer)
    4. trainer to run RL traiing evaluation pipeline


<img src="https://tianshou.org/en/stable/_images/pipeline.png">



In [5]:
import gymnasium as gym 
import tianshou as ts 

env = gym.make('CartPole-v1')
# CartPole-v1 includes a cart carrying a pole moving on a track
# a simple environmetn with a discrete action space
# you haave to identify whether the action space is continuous or discrete and apply suited algorithm

# state is the position of the cart   , the velocity of the cart , the anglel of the pole and the velocity of the tip of the pole
# action can only be of [0,1,2] for moving cart left, no move and right
# reward : each timestep you last you will receove a+1 reward
#done if Cartpole is out-of-range or timeout (the pole is more than 15 degrees from vertical or the cart moves more han 2.4 from the center or you last over 200 timesteps)
# info , extra info from environment silmuation

train_envs = gym.make('CartPole-v1')
test_envs = gym.make('CartPole-v1')




Tianshou supports 4 types of verctorized environemtn wrapper
* dummyvectorenv, the sequantional version using a single-thread for-loop
* subprocvectorenv : use python multiprocessing and pipe for concurrent execution
* shemvectorenv : usee share memry instead of pipe based on subprocvecotenv
* rayvectorenv : useray for concurrent activities and is currentlt the only choice for parallel simulation in a clustiner with multiple machines, it can used as follows


In [6]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v1') for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make('CartPole-v1') for _ in range(100)])

#setup 10 env in train_env and 100 env in test_envs




In [ ]:
# can also try super-fast vectorized environment EnvPool
import envpool
train_envs = envpool.make_gymnasium("CartPole-v1", num_envs=10)
test_envs = envpool.make_gymnasium("CartPole-v1", num_envs=100)

#https://github.com/sail-sg/envpool/


ModuleNotFoundError: No module named 'envpool'

In [15]:
import torch, numpy as np 
from torch import nn 

class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape),128), nn.ReLU(inplace=True),
            nn.Linear(128,128), nn.ReLU(inplace=True),
            nn.Linear(128,128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch,-1))
        return logits , state 

state_shape = env.observation_space.shape or env.observation_space.n 
action_shape = env.action_space.shape or env.action_space.n 
net = Net(state_shape, action_shape)
optim = torch.optim.Adam(net.parameters(), lr=1e-3)


you can also use pre-defined MLP networks in common, discrete and continuous. the rules of self-defined networks are 
1. input : observation obs (may be numpy.ndarray, torch.Tensor), hidden state provided by the environment
2. output : some logits , the next hidden state the logits could be a tuple instead of a Torch.tensor or some other useful variables or results during the policy forwarding procedure. it depends on how the polict class process the network output.


In [16]:
# setup poolicy
# we use the defined net and optim above with extra policy hyper-parameters to define a policy Here we deffine a DQN policy with a target network

policy = ts.policy.DQNPolicy(
    model=net,
    optim=optim,
    action_space=env.action_space,
    discount_factor=0.9,
    target_update_freq=320
)

# setup collector

* collector allows the polict to interact with different type of environments
* in each step, the collector will let the polict perform (at least) a specified numbers of steps or episodes and stoer the data in a replay buffer
* notce that wehn setup a collector, VectorReplapyBuffer to be used in vectorized envrionment scenarios, adn the number of buffers in the following case 10 is preferred to be set as the nuumber of environments


In [21]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

the main function of collector is the collect function which can be summarized in the following lines
result = self.policy(self.data, last_state)     #the agent predicts the batch action from batch observation
act = to_numpy(result.act)
self.data.update(act=act) #update the data with new action/policy
result = self.env.step(act, ready_env_ids)
obs_next, rew, done, info = result 
self.data.update(obs_next=obs_next, rew=rew, done=done, info=info)  #update the data with new state/reward/done/info

#self is not defined

NameError: name 'self' is not defined

# Train policy with a Trainer 

* tianshou provides OnpolicyTrainer, OffpolicyTrainer, OfflineTrainer . 
* the trainer will automatically stop training when the policy reaches the stop condition stop_fn on test collector 
* Since DQN is an off-policy algorithm, we use the OffpolicyTraine as follows



In [23]:
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=10, step_per_epoch=1000, step_per_collect=10,
    update_per_step=0.1,episode_per_test=100,batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn = lambda mean_rewards: mean_rewards >= env.spec.reward_threshold
).run()

print(f'finished training! use {result["duration"]}')

Epoch #1: 1001it [00:01, 637.63it/s, env_step=1000, gradient_step=100, len=21, n/ep=4, n/st=10, rew=21.00]


Epoch #1: test_reward: 14.870000 ± 1.764398, best_reward: 14.870000 ± 1.764398 in #1


Epoch #2: 1001it [00:00, 1093.64it/s, env_step=2000, gradient_step=200, len=20, n/ep=1, n/st=10, rew=20.00]


Epoch #2: test_reward: 33.290000 ± 47.105689, best_reward: 33.290000 ± 47.105689 in #2


Epoch #3: 1001it [00:00, 1143.12it/s, env_step=3000, gradient_step=300, len=9, n/ep=0, n/st=10, rew=9.25]


...

Epoch #9: 1001it [00:00, 1094.87it/s, env_step=9000, gradient_step=900, len=22, n/ep=0, n/st=10, rew=22.00]


Epoch #9: test_reward: 9.410000 ± 0.895489, best_reward: 33.290000 ± 47.105689 in #2


Epoch #10: 1001it [00:00, 1099.80it/s, env_step=10000, gradient_step=1000, len=15, n/ep=2, n/st=10, rew=15.50]


Epoch #10: test_reward: 10.680000 ± 1.248038, best_reward: 33.290000 ± 47.105689 in #2


TypeError: 'InfoStats' object is not subscriptable

the meaning of each parameter is as follows 
* max_epoch: the maximum of epochs for training . the training process might be finished before reaching the max_epoch
* step_per_epoch : the number of env step collected per epoch
* step_per_collect: the number of transition of collector would collect before the network update. for example the code above means "collect 10 transitiotions and do one policy network update"
* episode_per_test: the number of episodes for one policy evaluation
* batch_size = the batch size of sample data, which is going to feed in the policy network
* train_fn : a function receives the current number of epoch and step index, and performs some opertions at the beginning of training in this epoch. 
* test_fn : a function receives the average undscounted returns of the testing result, return a boolean which indicaetd whether goal reached


In [1]:
# the trainer supports tensorbaord for logging
from torch.utils.tensorboard import SummaryWriter 
from tianshou.utils import TensorboardLogger
writer = SummaryWriter('log/dqn')
logger = TensorboardLogger(writer)

pass the logger into the trainer.

the training result will be recorded into the tensorboard

it shows that within approximately 4 seconds , we finished training a DQN agent on CartPole. The mean returns over 100 consecutive episodes 199.03

## save/load policy

policy inherits the class torch.nn.Module, so since the policy inhertis that, saving and loading the policy are exectly thee same as a torch.module

In [ ]:
torch.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(torch.load('dqn.pth'))

# Watch the agent's performance

`Collector` supports rendering. Here is the example of watching the agent's performance in 35 FPS


In [ ]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35)

#can see action generated by trained agent
action = policy(Batch(obs=np.array([obs]))).act[0]



# Train a Policy with Customised Codes

if you dont want to use provided trainer and customize it, tianshou supports user-defined training code.

In [27]:
# pre-collect at least 5000 transitions with random action before training
train_collector.collect(n_step=5000, random=True)

policy.set_eps(0.1)
for i in range(int(1e6)):  # total step
    collect_result = train_collector.collect(n_step=10)

    # once if the collected episodes' mean returns reach the threshold,
    # or every 1000 steps, we test it on test_collector
    if collect_result['rews'].mean() >= env.spec.reward_threshold or i % 1000 == 0:
        policy.set_eps(0.05)
        result = test_collector.collect(n_episode=100)
        if result['rews'].mean() >= env.spec.reward_threshold:
            print(f'Finished training! Test mean returns: {result["rews"].mean()}')
            break
        else:
            # back to training eps
            policy.set_eps(0.1)

    # train policy with a sampled batch data from buffer
    losses = policy.update(64, train_collector.buffer)

TypeError: 'CollectStats' object is not subscriptable

SyntaxError: invalid syntax (269858882.py, line 97)

In [43]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tianshou.policy import DQNPolicy
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# Hyperparameters
lr = 1e-3
gamma = 0.9
n_step = 3
target_update_freq = 320
train_num = 8
test_num = 8
buffer_size = 20000
batch_size = 64
eps_train = 0.1
eps_test = 0.05
num_epochs = 10
step_per_epoch = 1000
collect_per_step = 10
update_per_collect = 10
episode_per_test = 10

# Make environments
train_envs = DummyVectorEnv([lambda: gym.make('CartPole-v1') for _ in range(train_num)])
test_envs = DummyVectorEnv([lambda: gym.make('CartPole-v1') for _ in range(test_num)])
train_envs.seed(seed)
test_envs.seed(seed)

# Define the network
class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_shape)
        )

    def forward(self, obs, state=None, info={}):
        if isinstance(obs, dict):
            obs = obs['obs']
        obs = torch.tensor(obs, dtype=torch.float32)
        return self.model(obs), state

# Create the policy
env = gym.make('CartPole-v1')
state_shape = env.observation_space.shape
action_shape = env.action_space.n
net = Net(state_shape, action_shape)
optimizer = optim.Adam(net.parameters(), lr=lr)

policy = DQNPolicy(
    model=net,
    optim=optimizer,
    action_space=env.action_space,
    discount_factor=gamma,
    estimation_step=n_step,
    target_update_freq=target_update_freq
)

# Collectors
train_collector = Collector(
    policy,
    train_envs,
    VectorReplayBuffer(buffer_size, len(train_envs))
)
test_collector = Collector(policy, test_envs)

# Initial random data
train_collector.collect(n_step=1000, reset_before_collect=True)

# Manual training loop
for epoch in range(num_epochs):
    policy.set_eps(eps_train)
    train_collector.collect(n_step=step_per_epoch)

    # Start training
    for _ in range(update_per_collect):
        batch = train_collector.buffer.sample(batch_size)
        policy.update(batch)

    # Set the policy to evaluation mode for testing
    policy.set_eps(eps_test)
    result = test_collector.collect(n_episode=episode_per_test)
    print(f"Epoch #{epoch + 1}: test_reward: {result['rews'].mean():.2f}, length: {result['lens'].mean():.2f}")

# Save model
torch.save(policy.state_dict(), 'dqn_cartpole_manual.pth')

# Render one episode with the trained policy
env = gym.make('CartPole-v1', render_mode='human')
policy.eval()
policy.set_eps(0.01)
collector = Collector(policy, env)
collector.collect(n_episode=1)
collector.close()


TypeError: BasePolicy.update() missing 1 required positional argument: 'buffer'